# 0.0. Intruducao

## 0.1. Biblíotecas

In [15]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

## 0.2. Loading dos Dados

In [2]:
df = pd.read_csv( 'train_users_2.csv', low_memory = True )
df_sessions = pd.read_csv( 'sessions.csv', low_memory= True )

## 0.3. Funcoes Auxiliares

# 1.0. Descricao dos Dados

In [28]:
df1 = df.copy()
df1_sessions = df_sessions

## 1.1. Dimensionalidade

In [29]:
print( 'Número de linhas: {}'.format( df1.shape[0] ) )
print( 'Número de colunas: {}'.format( df1.shape[1] ) )

Número de linhas: 213451
Número de colunas: 16


In [30]:
print( 'Número de linhas: {}'.format( df1_sessions.shape[0] ) )
print( 'Número de colunas: {}'.format( df1_sessions.shape[1] ) )

Número de linhas: 10567737
Número de colunas: 6


## 1.2. Data type

In [31]:
df1.dtypes

id                          object
date_account_created        object
timestamp_first_active       int64
date_first_booking          object
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
country_destination         object
dtype: object

In [32]:
df1_sessions.dtypes

user_id           object
action            object
action_type       object
action_detail     object
device_type       object
secs_elapsed     float64
dtype: object

In [33]:
aux = df1[df1['age'].isna()]
aux['country_destination'].value_counts( normalize=True )

NDF      0.768428
US       0.156529
other    0.028617
FR       0.014888
IT       0.009081
GB       0.006251
ES       0.006171
CA       0.003989
DE       0.002387
NL       0.001818
AU       0.001171
PT       0.000671
Name: country_destination, dtype: float64

## 1.2. Check Na

### 1.2.1. Check df1

In [34]:
#porcentagem dos dados faltantes por feature
df1.isna().sum() / len( df1 )

id                         0.000000
date_account_created       0.000000
timestamp_first_active     0.000000
date_first_booking         0.583473
gender                     0.000000
age                        0.412226
signup_method              0.000000
signup_flow                0.000000
language                   0.000000
affiliate_channel          0.000000
affiliate_provider         0.000000
first_affiliate_tracked    0.028414
signup_app                 0.000000
first_device_type          0.000000
first_browser              0.000000
country_destination        0.000000
dtype: float64

In [35]:
# remover missing values
#df1 = df1.dropna()

#date_first_booking
date_first_booking_max = pd.to_datetime( df1['date_first_booking'] ).max().strftime( '%Y-%m-%d' )
df1['date_first_booking'] = df1['date_first_booking'].fillna( date_first_booking_max )

#age
avg_age = df1['age'].mean().astype( int )
df1['age'] = df1['age'].fillna( avg_age )

#first_affiliate_tracked
df1 = df1[~df1['first_affiliate_tracked'].isna()]

df1.shape

(207386, 16)

### 1.2.2. Check df1_sessions

In [36]:
#porcentagem dos dados faltantes por feature
df1_sessions.isna().sum() / len( df1_sessions )

user_id          0.003264
action           0.007535
action_type      0.106570
action_detail    0.106570
device_type      0.000000
secs_elapsed     0.012872
dtype: float64

In [37]:
# drop na

#user_id - 0.3%
df1_sessions = df1_sessions[~df1_sessions['user_id'].isna()]

#action - 0.7%
df1_sessions = df1_sessions[~df1_sessions['action'].isna()]

#action_type - 10.6%
df1_sessions = df1_sessions[~df1_sessions['action_type'].isna()]

#action_detail - 10.6%
df1_sessions = df1_sessions[~df1_sessions['action_detail'].isna()]

#secs_elepsed - 1.2%
df1_sessions = df1_sessions[~df1_sessions['secs_elapsed'].isna()]

## 1.3. Change Types

In [7]:
#date_account_created
df1['date_account_created'] = pd.to_datetime( df['date_account_created'] )

#timestamp_first_active
df1['timestamp_first_active'] = pd.to_datetime( df['timestamp_first_active'], format = "%Y%m%d%H%M%S" )

#date_first_booking
df1['date_first_booking'] = pd.to_datetime( df['date_first_booking'] )

#age
df1['age'] = df1['age'].astype( int )

In [38]:
df1.sample().T

,99211
id,78xvqndnf8
date_account_created,2013-08-14
timestamp_first_active,20130814051212
date_first_booking,2013-08-14
gender,-unknown-
age,49.0
signup_method,basic
signup_flow,0
language,en
affiliate_channel,direct


## 1.4. Check Balancemento dos Dados

In [9]:
df1['country_destination'].value_counts( normalize=True )

US       0.708864
other    0.110854
FR       0.054085
IT       0.029763
GB       0.025935
ES       0.024864
CA       0.015696
DE       0.012454
NL       0.008801
AU       0.006366
PT       0.002318
Name: country_destination, dtype: float64

## 1.5. Estatistica Descritiva

In [10]:
atributos_num = df1.select_dtypes( include = [int, 'float64'] )
atributos_cat = df1.select_dtypes( exclude = [int, 'float64'] )
atributos_time = df1.select_dtypes( include = ['datetime64[ns]'] )

### 1.5.1. Atributos Numericos

In [12]:
# Tendencia Central - mean, median
tc1 = pd.DataFrame( atributos_num.apply( np.mean ) ).T
tc2 = pd.DataFrame( atributos_num.apply( np.median ) ).T

# Tendencia de dispersao - std, min, max, range, skew, kurtosis
td1 = pd.DataFrame( atributos_num.apply( np.std ) ).T
td2 = pd.DataFrame( atributos_num.apply( min ) ).T
td3 = pd.DataFrame( atributos_num.apply( max ) ).T
td4 = pd.DataFrame( atributos_num.apply( lambda x: x.max() - x.min() ) ).T
td5 = pd.DataFrame( atributos_num.apply( lambda x: x.skew() ) ).T
td6 = pd.DataFrame( atributos_num.apply( lambda x: x.kurtosis() ) ).T

# concat
ct = pd.concat( [td2, td3, td1, tc1, tc2, td4, td5, td6] ).T.reset_index()
ct.columns = ['atributos', 'min', 'max', 'range', 'media', 'mediana', 'std', 'skew', 'kurtosis']
ct

,atributos,min,max,range,media,mediana,std,skew,kurtosis
0,age,2.0,2014.0,146.035312,47.857593,33.0,2012.0,13.250136,175.132153
1,signup_flow,0.0,25.0,6.533959,2.436857,0.0,25.0,2.807117,6.439075
